In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])

# Setup


In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
)

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Play")):
    client.collections.delete("Play")

client.collections.create(
    "Play",
    vector_config=Configure.Vectors.self_provided(),

    multi_tenancy_config=Configure.multi_tenancy(True)

    # multi_tenancy_config=Configure.multi_tenancy(
    #     enabled=True,
    #     auto_tenant_creation=True, #Assign to non-existant tenant will create
    #     auto_tenant_activation=True
    # )
)

## Create tenants
> tenant name – must be made of alphanumeric characters (a-z, A-Z, 0-9), underscore (_), and hyphen (-), with a length between 1 and 64 characters'


In [ ]:
from weaviate.classes.tenants import Tenant

play = client.collections.use("Play")

play.tenants.create([
    Tenant(name="ten_A"),
    Tenant(name="ten_B"),
    Tenant(name="ten_C"),
    Tenant(name="ten_D"),
])

## List Tenants

In [ ]:
play.tenants.get()

In [ ]:
play.tenants.exists("ten_E")

## Access Tenants

In [ ]:
# this will fail – multi-tenant collections require us to use tenants
play.aggregate.over_all()

In [ ]:
tenA = play.with_tenant("ten_A")

tenA.aggregate.over_all()

### Insert data

In [ ]:
# tenA = client.collections.use("Play").with_tenant("ten_A")
play = client.collections.use("Play")
tenA = play.with_tenant("ten_A")

tenA.data.insert_many([
    {
       "title": "A book about vector databases"
    },
    {
       "title": "Tutorial for multimodal collections"
    },
])

tenA.aggregate.over_all()

### Query Example

In [ ]:
from weaviate.classes.query import Filter

tenA = client.collections.use("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

### Delete Tenants

In [ ]:
play.tenants.remove(["ten_D"])
play.tenants.get()

### Update Tenants – Active & Inactive & Offloaded
Tenants can be:
* `Active` (default) - active tenants use  `HOT` resources (RAM)
* `Inacative` - inactive tenants cannot be searched on, their index is not loaded into memory, they don't use (RAM)
* `Offloaded` - offloaded tenants are moved to a cloud storage

> Tenant offloading, requires an extra configuration, which is out of scope for this workshop.<br/>
> You can learn more from [How-to: Configure - Tenant Offloading](https://weaviate.io/developers/weaviate/configuration/tenant-offloading)

### Deactivate – make tenant `Inactive`

In [ ]:
from weaviate.classes.tenants import Tenant, TenantActivityStatus

play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.INACTIVE),
])

play.tenants.get()

**Cannot search `Inactive` tenants**

In [ ]:
# tenA = client.collections.use("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

### Activate - make tenant `Active`

> You can't query an inactive tenant, but you can activate it.

In [ ]:
play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.ACTIVE),
])

play.tenants.get()

In [ ]:
response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

### Offload - make tenant `offloaded`

> Tenant offloading, requires an extra configuration, which is out of scope for this workshop.<br/>
> You can learn more from [How-to: Configure - Tenant Offloading](https://weaviate.io/developers/weaviate/configuration/tenant-offloading)

In [ ]:
# play.tenants.update([
#     Tenant(name="ten_A", activity_status=TenantActivityStatus.OFFLOADED),
# ])

# play.tenants.get()

## Clean up

In [ ]:
client.collections.delete("Play")

## Don't close yet...

> You can try again with `auto_tenant_creation=True` and `auto_tenant_activation=True`

## Close the client

In [ ]:
client.close()